In [1]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging
import torch

# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
    
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [2]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder=False,
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=500, #2048 is Llama's limit
    max_output_tokens=200,
    num_input_prompts=10,
    save_outputs=True,
    decode_token_to_text=True,
    gpu_list=[0,1,2,3],
    num_processes=4,
    batching_options={
        "batch_size___fixed_batching":16,
        "adaptive_batching": False,
        "adaptive_max_tokens": 3000,
        "max_batch_size___adaptive_batching": 100
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD" #FULL_SHARD doesn't work
    },
    query_rate=1,
    decoder_temperature=1.0, # if 0: deterministic, if >0 do_sample is activated. NB: needs to be a float.. Higher Temperature → Longer Outputs (sometimes)If stopping criterion is based on tokens (e.g., EOS token), high temperature might delay reaching it.

    fp_precision="float32",
    quantization_config={
        "quantization": True,
        "cached_flops_for_quantised_models": 5172720640000, # THIS IS NEEDED UNTIL I CAN WORK OUT HOW TO COMPUTE FLOPS OF QUANITSED MODELS
        "load_in_8bit": False,
        "load_in_4bit": True,
        #"llm_int8_threshold"=6.0,
        "llm_int8_enable_fp32_cpu_offload": False,
        "llm_int8_has_fp16_weight": False,
    },
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# function to run the whole experiment workflow.
def run_experiment_workflow():
    
    # run experiment
    runner = ExperimentRunner(experiment_config, prompts)
    
    runner.run_torch()
    
    # If not main process, exit cleanly after run
    if not runner.accelerator.is_main_process:
        return  # do NOT proceed further

    # Only main process continues
    if runner.accelerator.is_main_process:
         # aggregate results
        runner.aggregate_results()
        # save results
        runner.save_experiment_results()  
        
    # could add in script here to delete JSON files if storage an issue
    
    runner.teardown()

    return

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes) 
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()                 

Launching training on 4 GPUs.
Accelerator set up
Unique experiment id: 0268


`low_cpu_mem_usage` was None, now default to True since model is quantized.


TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using pytorch, with precision float32
Entering wait barrier: after load_model_tokenizer


`low_cpu_mem_usage` was None, now default to True since model is quantized.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


wait_for_everyone completed within 10 seconds for after load_model_tokenizer.
Exiting wait barrier: after load_model_tokenizer
Original generate method saved.
Model and tokenizer prepared
Entering wait barrier: after model preparation


...

INFO:classes.experiment_runner:[Process 3707751][GPU 2] saved its energy metrics.


INFO:classes.experiment_runner:Process 0: Energy metrics combined successfully.
INFO:classes.experiment_runner:[Process 3707749][GPU 0] saved its energy metrics.


All local process energy metrics saved
Experiment finished
Destroyed process group successfully


INFO:classes.experiment_runner:Aggregating per-process energy metrics from disk.
INFO:classes.experiment_runner:Aggregated global energy results successfully from disk.
INFO:classes.experiment_runner:Saving experiment results
INFO:classes.experiment_runner:Experiment results saved to results/text_generation_results.json


Starting teardown process...
Process group not initialized.
Emptying CUDA cache...
Running garbage collection...
Teardown process complete.
